In [1]:
!pip install nltk
!pip install scikit-learn
!pip install openpyxl
!pip install pickle

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [2]:
# Instalando os pacotes necessários
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/victorcarvalho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/victorcarvalho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
dataframe = pd.read_excel("NORAM-Data.xlsx")

In [6]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Definindo método de pré-processamento de texto
def preprocess_text(text):
    tokens = word_tokenize(text)  
    tokens = [word for word in tokens if word.isalnum()]  
    tokens = [word.lower() for word in tokens] 

    tokens = [word for word in tokens if word not in stop_words] 

    stemmed_tokens = [stemmer.stem(word) for word in tokens]  

    return ' '.join(stemmed_tokens)


dataframe = dataframe.dropna(subset=['Business Unit'])

# Aplicando o método nas colunas de interesse
columns_to_preprocess = ['Supplier Name', 'Invoice Source', 'Country Name', 'Legal Entity', 'Business Unit', 'GL Description']
for column in columns_to_preprocess:
    dataframe[column] = dataframe[column].apply(preprocess_text)

# Combine as colunas de texto
combined_text_columns = dataframe[columns_to_preprocess].apply(lambda row: ' '.join(row), axis=1)


# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(combined_text_columns, dataframe['Level 1'], test_size=0.2, random_state=42)

# Vetorizando o texto ponderando pela frequência de ocorrência
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [7]:
# Treinando modelo Multinomial Naive Bayes
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

# Prevendo os valores do conjunto de teste
y_pred = nb_classifier.predict(X_test_tfidf)

# Avaliando o modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Acurácia do modelo: {accuracy}')
print(f'Relatório de classificação:\n{report}')

Acurácia do modelo: 0.89761455769702
Relatório de classificação:
                                                       precision    recall  f1-score   support

                                   Energy & Utilities       0.86      0.68      0.76       173
                                      Human Resources       0.97      0.99      0.98     31886
                                            Logistics       0.90      0.97      0.94      1387
                                        Manufacturing       0.67      0.67      0.67       819
                                Professional Services       0.87      0.88      0.87      7158
R&D Equipment (incl. Equipment Services and Supplies)       0.60      0.85      0.70      3012
                             Real Estate & Facilities       0.82      0.94      0.87     12975
                            Sales, Marketing & Events       0.72      0.63      0.67      2452
                                   Technology/Telecom       0.95      0.94     

In [8]:
# Criação do classificador SVM
svm_classifier = SVC(kernel='linear', random_state=42)  # Aqui, linear é um kernel comum para textos

# Treinamento do modelo SVM com os dados de treino
svm_classifier.fit(X_train_tfidf, y_train)

# Previsões com dados de teste
y_pred = svm_classifier.predict(X_test_tfidf)

# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Acurácia do modelo SVM: {accuracy}')
print(f'Relatório de classificação:\n{report}')

Acurácia do modelo SVM: 0.9168877382512438
Relatório de classificação:
                                                       precision    recall  f1-score   support

                                   Energy & Utilities       0.76      0.78      0.77       173
                                      Human Resources       0.99      0.99      0.99     31886
                                            Logistics       0.93      0.97      0.95      1387
                                        Manufacturing       0.85      0.70      0.77       819
                                Professional Services       0.87      0.93      0.90      7158
R&D Equipment (incl. Equipment Services and Supplies)       0.59      0.88      0.71      3012
                             Real Estate & Facilities       0.87      0.94      0.90     12975
                            Sales, Marketing & Events       0.77      0.75      0.76      2452
                                   Technology/Telecom       0.96      0.9

In [9]:
# Criando o classificador KNN
knn_classifier = KNeighborsClassifier(n_neighbors=5)  # Aqui, 'n_neighbors' é o número de vizinhos que o KNN considerará

# Treinando o modelo KNN
knn_classifier.fit(X_train_tfidf, y_train)

# Fazendo previsões com dados de teste
y_pred = knn_classifier.predict(X_test_tfidf)

# Avaliando o modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Acurácia do modelo KNN: {accuracy}')
print(f'Relatório de classificação:\n{report}')

Acurácia do modelo KNN: 0.9210960888974121
Relatório de classificação:
                                                       precision    recall  f1-score   support

                                   Energy & Utilities       0.78      0.92      0.84       173
                                      Human Resources       0.99      0.99      0.99     31886
                                            Logistics       0.94      0.97      0.95      1387
                                        Manufacturing       0.78      0.79      0.78       819
                                Professional Services       0.92      0.95      0.93      7158
R&D Equipment (incl. Equipment Services and Supplies)       0.64      0.69      0.66      3012
                             Real Estate & Facilities       0.88      0.91      0.90     12975
                            Sales, Marketing & Events       0.82      0.79      0.81      2452
                                   Technology/Telecom       0.96      0.9

In [10]:
# Treinamento do modelo Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train_tfidf, y_train)

# Predição no conjunto de teste
y_pred = random_forest.predict(X_test_tfidf)

# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Acurácia do modelo Random Forest: {accuracy}')
print(f'Relatório de classificação:\n{report}')

Acurácia do modelo Random Forest: 0.9267960068612097
Relatório de classificação:
                                                       precision    recall  f1-score   support

                                   Energy & Utilities       0.80      0.92      0.86       173
                                      Human Resources       0.99      0.99      0.99     31886
                                            Logistics       0.93      0.97      0.95      1387
                                        Manufacturing       0.85      0.78      0.82       819
                                Professional Services       0.93      0.95      0.94      7158
R&D Equipment (incl. Equipment Services and Supplies)       0.61      0.89      0.72      3012
                             Real Estate & Facilities       0.87      0.95      0.91     12975
                            Sales, Marketing & Events       0.82      0.82      0.82      2452
                                   Technology/Telecom       0.9

In [11]:
new_description = ['3D SYSTEMS INC', 'OCR', 'United States of America', 'Meta Platforms Technologies, LLC', 'RL', 'Deploy and create new servers']

new_description = [preprocess_text(desc) for desc in new_description]
print(new_description)
new_description_tfidf = tfidf_vectorizer.transform(new_description)
print(new_description_tfidf)

predicted_category_nb = nb_classifier.predict(new_description_tfidf)
predicted_category_knn = knn_classifier.predict(new_description_tfidf)
predicted_category_svm = svm_classifier.predict(new_description_tfidf)
predicted_category_rf = random_forest.predict(new_description_tfidf)


print(f'A categoria prevista para a nova descrição é:')
print(f'Naive Bayes -> {predicted_category_nb[0]}')
print(f'K-Nearest Neighbors -> {predicted_category_knn[0]}')
print(f'Support Vector Machine -> {predicted_category_nb[0]}')
print(f'Random Forest -> {predicted_category_nb[0]}')

['3d system inc', 'ocr', 'unit state america', 'meta platform technolog llc', 'rl', 'deploy creat new server']
  (0, 867)	0.9815461926092428
  (0, 435)	0.19122518603546781
  (1, 636)	1.0
  (2, 930)	0.575161995669633
  (2, 840)	0.5784420723602063
  (2, 37)	0.5784405307902922
  (3, 879)	0.785104563034223
  (3, 688)	0.2846511545880288
  (3, 577)	0.28459024436119534
  (3, 533)	0.4707365910028439
  (4, 764)	1.0
  (5, 620)	1.0
A categoria prevista para a nova descrição é:
Naive Bayes -> Technology/Telecom
K-Nearest Neighbors -> Technology/Telecom
Support Vector Machine -> Technology/Telecom
Random Forest -> Technology/Telecom


In [12]:
# with open('nb_model.pkl', 'wb') as arquivo:
#     pickle.dump(nb_classifier, arquivo)

In [13]:
# with open('svm_model.pkl', 'wb') as arquivo:
#     pickle.dump(svm_classifier, arquivo)

In [14]:
# with open('knn_model.pkl', 'wb') as arquivo:
#     pickle.dump(knn_classifier, arquivo)

In [15]:
# with open('tfidf_vectorizer.pkl', 'wb') as arquivo:
#     pickle.dump(tfidf_vectorizer, arquivo)